### Experiment
Run Hebbian pruning with non-binary activations.

### Motivation
Attempt pruning given intuition offered up in "Memory Aware Synapses" paper:
     * The weights with higher coactivations computed as $x_i \times x_j$
     have a greater effect on the L2 norm of the layers output. Here $x_i$ and $x_j$ are
     the input and output activations respectively. 

In [113]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [114]:
import sys
sys.path.append("../../")

In [115]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import glob
import tabulate
import pprint
import click
import numpy as np
import pandas as pd
from ray.tune.commands import *
from nupic.research.frameworks.dynamic_sparse.common.browser import *

In [128]:
base = 'gsc-trials-2019-10-02'
exps = [
    os.path.join(base, exp) for exp in [
        'gsc-BaseModel', 
        'gsc-Heb',
        'gsc-SET',
        'gsc-Static',
    ]
]
    
paths = [os.path.expanduser("~/nta/results/{}".format(e)) for e in exps]
for p in paths:
    print(os.path.exists(p), p)
df = load_many(paths)

True /Users/mcaporale/nta/results/gsc-trials-2019-10-02/gsc-BaseModel
True /Users/mcaporale/nta/results/gsc-trials-2019-10-02/gsc-Heb
True /Users/mcaporale/nta/results/gsc-trials-2019-10-02/gsc-SET
True /Users/mcaporale/nta/results/gsc-trials-2019-10-02/gsc-Static
exp_name /Users/mcaporale/nta/results/gsc-trials-2019-10-02/gsc-BaseModel/experiment_state-2019-10-03_02-48-06.json
exp_name /Users/mcaporale/nta/results/gsc-trials-2019-10-02/gsc-Heb/experiment_state-2019-10-03_02-48-06.json
exp_name /Users/mcaporale/nta/results/gsc-trials-2019-10-02/gsc-SET/experiment_state-2019-10-03_02-48-06.json
exp_name /Users/mcaporale/nta/results/gsc-trials-2019-10-02/gsc-Static/experiment_state-2019-10-03_02-48-06.json


In [129]:
def leqauls(l1, l2):
    """
    See if two list are the same.
    """
    if len(l1) != len(l2):
        return False
    for i1, i2 in zip(l1, l1):
        if i1 != i2:
            return False
    
    return True


In [130]:
df

,Experiment Name,train_acc_max,train_acc_max_epoch,train_acc_min,train_acc_min_epoch,train_acc_median,train_acc_last,val_acc_max,val_acc_max_epoch,val_acc_min,...,optim_alg,test_noise,weight_decay,hebbian_grow,hebbian_prune_perc,moving_average_alpha,on_perc,prune_methods,use_binary_coactivations,weight_prune_perc
0,0_model=BaseModel,0.955327,29,0.673958,0,0.943755,0.955327,0.964314,27,0.900561,...,SGD,False,0.01,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1_model=BaseModel,0.954985,26,0.649107,0,0.940997,0.952202,0.963512,20,0.889735,...,SGD,False,0.01,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2_model=BaseModel,0.953423,27,0.649497,0,0.942535,0.952153,0.969928,20,0.899759,...,SGD,False,0.01,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3_model=BaseModel,0.954692,26,0.666146,0,0.941095,0.953569,0.966720,25,0.882919,...,SGD,False,0.01,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4_model=BaseModel,0.957035,25,0.641246,0,0.944415,0.955766,0.965517,26,0.886127,...,SGD,False,0.01,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,"0_model=DSNNMixedHeb,moving_average_alpha=0.6,...",0.955375,27,0.646763,0,0.942755,0.953862,0.962711,13,0.888132,...,SGD,False,0.01,True,0.3,0.6,None-None-0.4-None,None-None-dynamic-linear-None,False,None
6,"1_model=DSNNMixedHeb,moving_average_alpha=0.6,...",0.951323,25,0.663802,0,0.937482,0.948443,0.965517,27,0.876103,...,SGD,False,0.01,True,0.3,0.6,None-None-0.1-None,None-None-dynamic-linear-None,False,None
7,"2_model=DSNNMixedHeb,moving_average_alpha=0.6,...",0.953374,29,0.643004,0,0.941851,0.953374,0.965517,29,0.877306,...,SGD,False,0.01,True,0.3,0.6,None-None-0.4-None,None-None-dynamic-linear-None,False,None
8,"3_model=DSNNMixedHeb,moving_average_alpha=0.6,...",0.952690,24,0.686993,0,0.941949,0.951714,0.963512,29,0.891740,...,SGD,False,0.01,True,0.3,0.6,None-None-0.1-None,None-None-dynamic-linear-None,False,None
9,"4_model=DSNNMixedHeb,moving_average_alpha=0.6,...",0.954692,27,0.650327,0,0.943145,0.953423,0.963512,21,0.895750,...,SGD,False,0.01,True,0.3,0.6,None-None-0.4-None,None-None-dynamic-linear-None,False,None


In [131]:
# remove nans where appropriate
df['hebbian_prune_perc'] = df['hebbian_prune_perc'].replace(np.nan, 0.0, regex=True)
df['weight_prune_perc'] = df['weight_prune_perc'].replace(np.nan, 0.0, regex=True)

# distill certain values 
df['on_perc'] = df['on_perc'].replace('None-None-0.1-None', 0.1, regex=True)
df['on_perc'] = df['on_perc'].replace('None-None-0.4-None', 0.4, regex=True)
df['prune_methods'] = df['prune_methods'].replace('None-None-dynamic-linear-None', 'dynamic-linear', regex=True)

In [136]:
df

,Experiment Name,train_acc_max,train_acc_max_epoch,train_acc_min,train_acc_min_epoch,train_acc_median,train_acc_last,val_acc_max,val_acc_max_epoch,val_acc_min,...,optim_alg,test_noise,weight_decay,hebbian_grow,hebbian_prune_perc,moving_average_alpha,on_perc,prune_methods,use_binary_coactivations,weight_prune_perc
0,0_model=BaseModel,0.955327,29,0.673958,0,0.943755,0.955327,0.964314,27,0.900561,...,SGD,False,0.01,NaN,0.0,NaN,NaN,NaN,NaN,0.0
1,1_model=BaseModel,0.954985,26,0.649107,0,0.940997,0.952202,0.963512,20,0.889735,...,SGD,False,0.01,NaN,0.0,NaN,NaN,NaN,NaN,0.0
2,2_model=BaseModel,0.953423,27,0.649497,0,0.942535,0.952153,0.969928,20,0.899759,...,SGD,False,0.01,NaN,0.0,NaN,NaN,NaN,NaN,0.0
3,3_model=BaseModel,0.954692,26,0.666146,0,0.941095,0.953569,0.966720,25,0.882919,...,SGD,False,0.01,NaN,0.0,NaN,NaN,NaN,NaN,0.0
4,4_model=BaseModel,0.957035,25,0.641246,0,0.944415,0.955766,0.965517,26,0.886127,...,SGD,False,0.01,NaN,0.0,NaN,NaN,NaN,NaN,0.0
5,"0_model=DSNNMixedHeb,moving_average_alpha=0.6,...",0.955375,27,0.646763,0,0.942755,0.953862,0.962711,13,0.888132,...,SGD,False,0.01,True,0.3,0.6,0.4,dynamic-linear,False,0.0
6,"1_model=DSNNMixedHeb,moving_average_alpha=0.6,...",0.951323,25,0.663802,0,0.937482,0.948443,0.965517,27,0.876103,...,SGD,False,0.01,True,0.3,0.6,0.1,dynamic-linear,False,0.0
7,"2_model=DSNNMixedHeb,moving_average_alpha=0.6,...",0.953374,29,0.643004,0,0.941851,0.953374,0.965517,29,0.877306,...,SGD,False,0.01,True,0.3,0.6,0.4,dynamic-linear,False,0.0
8,"3_model=DSNNMixedHeb,moving_average_alpha=0.6,...",0.952690,24,0.686993,0,0.941949,0.951714,0.963512,29,0.891740,...,SGD,False,0.01,True,0.3,0.6,0.1,dynamic-linear,False,0.0
9,"4_model=DSNNMixedHeb,moving_average_alpha=0.6,...",0.954692,27,0.650327,0,0.943145,0.953423,0.963512,21,0.895750,...,SGD,False,0.01,True,0.3,0.6,0.4,dynamic-linear,False,0.0


In [45]:
df.columns

Index(['Experiment Name', 'train_acc_max', 'train_acc_max_epoch',
       'train_acc_min', 'train_acc_min_epoch', 'train_acc_median',
       'train_acc_last', 'val_acc_max', 'val_acc_max_epoch', 'val_acc_min',
       'val_acc_min_epoch', 'val_acc_median', 'val_acc_last', 'val_acc_all',
       'epochs', 'experiment_file_name', 'trial_time', 'mean_epoch_time',
       'batch_size_test', 'batch_size_train', 'data_dir', 'dataset_name',
       'debug_sparse', 'debug_weights', 'device', 'learning_rate', 'lr_gamma',
       'lr_scheduler', 'lr_step_size', 'model', 'momentum', 'network',
       'optim_alg', 'test_noise', 'weight_decay', 'hebbian_grow',
       'hebbian_prune_perc', 'moving_average_alpha', 'on_perc',
       'prune_methods', 'use_binary_coactivations', 'weight_prune_perc'],
      dtype='object')

In [56]:
df.shape

(35, 42)

In [59]:
df.iloc[34]

Experiment Name             9_model=SparseModel,on_perc=[None, None, 0.1, ...
train_acc_max                                                        0.952202
train_acc_max_epoch                                                        26
train_acc_min                                                        0.681086
train_acc_min_epoch                                                         0
train_acc_median                                                     0.941436
train_acc_last                                                       0.950395
val_acc_max                                                           0.96231
val_acc_max_epoch                                                          15
val_acc_min                                                          0.893745
val_acc_min_epoch                                                           0
val_acc_median                                                       0.957097
val_acc_last                                                    

In [81]:
df.groupby('model')["model"].count()

model
BaseModel        5
DSNNMixedHeb    20
SparseModel     10
Name: model, dtype: int64

In [78]:
# Did anything fail?
df[df["epochs"] < 30]["epochs"].count()

0

In [79]:
# helper functions
def mean_and_std(s):
    return "{:.3f} ± {:.3f}".format(s.mean(), s.std())

def round_mean(s):
    return "{:.0f}".format(round(s.mean()))

stats = ['min', 'max', 'mean', 'std']

def agg(columns, filter=None, round=3):
    if filter is None:
        return (df.groupby(columns)
             .agg({'val_acc_max_epoch': round_mean,
                   'val_acc_max': stats,                
                   'model': ['count']})).round(round)
    else:
        return (df[filter].groupby(columns)
             .agg({'val_acc_max_epoch': round_mean,
                   'val_acc_max': stats,                
                   'model': ['count']})).round(round)

In [143]:
type(np.nan)

float

In [150]:
df['on_perc'][0] is nan

False

## Dense Model

In [151]:
fltr = (df['model'] == 'BaseModel')
agg(['model'], fltr)

val_acc_max_epoch val_acc_max                     model
                 round_mean         min   max   mean    std count
model                                                            
BaseModel                24       0.964  0.97  0.966  0.003     5

## Prune via Hebbian

In [155]:
fltr = (df['on_perc'] == 0.4) & (df['hebbian_prune_perc'] == 0.3)
agg(['model'], fltr)

val_acc_max_epoch val_acc_max                      model
                    round_mean         min    max   mean    std count
model                                                                
DSNNMixedHeb                22       0.962  0.966  0.964  0.001     5

In [158]:
fltr = (df['on_perc'] == 0.1) & (df['hebbian_prune_perc'] == 0.3)
agg(['model'], fltr)

val_acc_max_epoch val_acc_max                      model
                    round_mean         min    max   mean    std count
model                                                                
DSNNMixedHeb                22       0.961  0.967  0.964  0.003     5

## SET

In [159]:
# 40% sparse 
fltr = (df['on_perc'] == 0.4) & (df['weight_prune_perc'] == 0.3)
agg(['model'], fltr)

val_acc_max_epoch val_acc_max                      model
                    round_mean         min    max   mean    std count
model                                                                
DSNNMixedHeb                26       0.962  0.968  0.966  0.002     5

In [160]:
# 10% sparse 
fltr = (df['on_perc'] == 0.1) & (df['weight_prune_perc'] == 0.3)
agg(['model'], fltr)

val_acc_max_epoch val_acc_max                      model
                    round_mean         min    max   mean    std count
model                                                                
DSNNMixedHeb                23        0.96  0.967  0.963  0.003     5

## Static Sparse

In [161]:
# 40% sparse 
fltr = (df['on_perc'] == 0.4) & (df['weight_prune_perc'] == 0.0)& (df['hebbian_prune_perc'] == 0.0)
agg(['model'], fltr)

val_acc_max_epoch val_acc_max                      model
                   round_mean         min    max   mean    std count
model                                                               
SparseModel                24       0.963  0.968  0.965  0.002     5

In [163]:
# 10% sparse 
fltr = (df['on_perc'] == 0.1) & (df['weight_prune_perc'] == 0.0)& (df['hebbian_prune_perc'] == 0.0)
agg(['model'], fltr)

val_acc_max_epoch val_acc_max                      model
                   round_mean         min    max   mean    std count
model                                                               
SparseModel                22       0.962  0.965  0.964  0.001     5